In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import os
import sys
import shutil
import splitfolders
import time
from PIL import Image
import math
import logging

In [9]:
IMG_SHAPE  = 224
batch_size = 32
random_seed = 9

In [10]:
# train, test split
splitfolders.ratio('/content/drive/MyDrive/SMOKE/', output="./splitted_data", ratio=(0.7, 0.3))

Copying files: 3804 files [01:15, 50.25 files/s] 


In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('./splitted_data/train/',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2662 images belonging to 2 classes.


In [12]:
training_set.class_indices

{'train_fire': 0, 'train_smoke': 1}

In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('./splitted_data/val/',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1142 images belonging to 2 classes.


In [14]:
# cnn
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=7, activation='relu', input_shape=[64,64,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 58, 58, 32)        4736      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 29, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 256)               1

In [16]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [17]:
history = cnn.fit(x = training_set, epochs = 15,validation_data=test_set, callbacks=[es])

Epoch 1/7
84/84 [==============================] - 80s 802ms/step - loss: 0.2455 - accuracy: 0.9053 - val_loss: 0.1841 - val_accuracy: 0.9291
Epoch 2/7
84/84 [==============================] - 66s 788ms/step - loss: 0.1131 - accuracy: 0.9628 - val_loss: 0.3065 - val_accuracy: 0.9247
Epoch 3/7
84/84 [==============================] - 67s 801ms/step - loss: 0.1218 - accuracy: 0.9651 - val_loss: 0.0834 - val_accuracy: 0.9764
Epoch 4/7
84/84 [==============================] - 66s 787ms/step - loss: 0.0960 - accuracy: 0.9647 - val_loss: 0.0644 - val_accuracy: 0.9729
Epoch 5/7
84/84 [==============================] - 67s 803ms/step - loss: 0.0632 - accuracy: 0.9771 - val_loss: 0.0723 - val_accuracy: 0.9729
Epoch 6/7
84/84 [==============================] - 66s 787ms/step - loss: 0.0632 - accuracy: 0.9831 - val_loss: 0.0423 - val_accuracy: 0.9834
Epoch 7/7
84/84 [==============================] - 69s 827ms/step - loss: 0.0646 - accuracy: 0.9823 - val_loss: 0.0730 - val_accuracy: 0.9729


In [18]:
start_time = time.time()

loss, accuracy = cnn.evaluate(training_set,batch_size=batch_size)
print('test_loss: %.2f' % (loss * 100.0))  
print('test_accuracy: %.2f' % (accuracy * 100.0))

logging.info("Evaluation Execution Time: " + time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))
logging.info("test_loss:" + str(loss))
logging.info("test accuracy:" + str(accuracy))

84/84 [==============================] - 49s 581ms/step - loss: 0.0607 - accuracy: 0.9797
test_loss: 6.07
test_accuracy: 97.97
